# Age Estimation and Gender Classification

In this assignment, you will train CNN models to estimate a person's age and gender by given a face image. Please read carefully all the instructions before you start to write your code.

**Your tasks**

You need train two CNN models:
- one is defined by you with a few restrictions and be trained from scratch, save it as `age_gender_A.h5`
- the other is to finetune a pretrained model, save it as `age_gender_B.h5`

**Dataset**

Your models will be trained and validated on a folder `train_val/` containing 5,000 labeled face images (size: 128 x 128), originated from the UTKFace dataset. During marking, your code will be tested on unseen test data. 

**Performance metric**

The metrics for measuring the performance on the test set are:
- age estimation: MAE (Mean Absolute Error)
- gender classification: accuracy

**Please use the GPU time wisely.**

Just be aware that there is some limit of free GPU usage (It is said the users can use up to 12 hours in row. But many people found they reached its limit far less than 12 hours.). Therefore, I would give you three suggestions to mimimise the risk of reaching the limit.

1. Make sure you have a stable internet connection.
2. Double check all the hyperparameters are good before you start to train the model.
3. According to my experience, each model should be trained in less than 2 hours. If much longer than that, you'd better consider adjusting the architecture.

## Setting Up: Mount the google drive
Mount your google drive to the notebook. 

Also don't forget to **enable GPU** before your training.


In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer 
%matplotlib inline


#Constants
data_dir = 'train_val/'
random_seed = 20
test_size = 0.2
batch_size = 128

## Visualize a few photos
It is always benificial to know your data well before you start. Here display a few (at least 20) images together with its corresponding age and gender from the `train_val/` folder to have a first impression of the dataset. You may also check what the size of the images are.

[(128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3), (128, 128, 3)]


In [3]:
#Function to extract stats from file name
def extract_age_gender(filename):
    age, gender = filename.split('_')[:2]
    age, gender = float(age), int(gender)
    return age, gender
  

image_shapes = []
plt.figure(figsize=(15, 15))

for i, file_name in enumerate(np.random.choice(os.listdir(data_dir), random_seed), start=1):
    plt.subplot(5, 5, i)
    #Read image
    img = plt.imread(os.path.join(data_dir, file_name))
    image_shapes.append(img.shape)
    plt.imshow(img)
    age, gender = extract_age_gender(file_name)
    plt.title(f'Age = {age}, Gender = {gender}')
    plt.axis('off')
    
print(image_shapes)

## Rearrange the dataset
You may do any arrangement for the dataset to suit your later process, such as splitting into training set and validation set, saving the gender labels and age some how, and so on.


In [4]:
import glob
import re

# Retrieve image file paths
file_paths = glob.glob(data_dir + '*.jpg')

# Check if dataset has enough samples
if len(file_paths) < 2:
    raise ValueError('Insufficient data for splitting. At least 2 samples are required.')

# Initialize empty lists for data
images = []
ages = []
genders = []
names = []

# Load images and extract labels
for file_path in file_paths:
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    age, gender = extract_age_gender(os.path.basename(file_path))
    images.append(image)
    names.append(file_path)
    ages.append(age)
    genders.append(gender)

# Convert data to numpy arrays
images = np.array(images, dtype=np.float32) / 255.0
ages = np.array(ages, dtype=np.float32)
genders = np.array(genders, dtype=np.float32)
names = np.array(names)

# Create dictionary for Pandas DataFrame
dict_labels = {
    'names': names,
    'ages': ages,
    'genders': genders
}

# Create DataFrame and display the first 5 rows
df = pd.DataFrame(dict_labels)
print(df.head(), '\n')



                                              names   ages  genders
0  train_val\100_1_0_20170112215032192.jpg.chip.jpg  100.0      1.0
1  train_val\100_1_2_20170112213615815.jpg.chip.jpg  100.0      1.0
2   train_val\10_0_0_20161220222308131.jpg.chip.jpg   10.0      0.0
3   train_val\10_0_0_20170110220255346.jpg.chip.jpg   10.0      0.0
4   train_val\10_0_0_20170110220644705.jpg.chip.jpg   10.0      0.0 



## STEP1: Data pre-processing
Now you need do some pre-processing before feeding data into a CNN network. You may consider:

1.	Rescale the pixel values (integers between 0 and 255) to [0,1]. **You must do this rescaling.** Otherwise the testing performance will be affected significantly, as the test images will be rescaling in this way. 
2.	Data augmentation.

**Don't rescale the age to [0,1].** Otherwise the testing performance will be affected significantly, as the original age is used in the testing stage. 

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# Rescaling the pixel values to [0, 1]
images = images/255.0

# Data augmentation
#Chollet, F. (2018) Deep Learning with Python. Setting used from reference
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = test_size
)

testdatagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = test_size
)

# Apply data augmentation on the training set
train_generator = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = 'names',
    y_col = ['ages', 'genders'],
    target_size = (128, 128),
    batch_size= batch_size,
    class_mode = 'raw',
    shuffle = True,
    subset = 'training')

# No data augmentation on the testing set
test_generator = testdatagen.flow_from_dataframe(
    dataframe = df,
    x_col = 'names',
    y_col = ['ages', 'genders'],
    target_size = (128, 128),
    batch_size= batch_size,
    class_mode = 'raw',
    shuffle = True,
    subset = 'validation')


Found 4000 validated image filenames.
Found 1000 validated image filenames.


## STEP2A: Build your own CNN network
Define your own CNN for classifying the gender and predicting the age. Though there are two tasks, you need **only one CNN model, but with two outputs** - you may search online for solution.

There are a few restrictions about your network as follows.
1.	The input size must be 128 x 128 x 3, which means you **should not resize** the original images. This is because my test code relies on this particular input size. Any other size will cause problem in the testing stage.
2.  Please treat the gender classification as a binary problem, i.e., **the output layer for the gender branch has only 1 unit**, instead of 2 (though it is correct to treat the gender classification as a mutli-class problem where class number is 2, the last layer has 2 units). This is because my test code only works for the 1-unit-last-layer gender branch setting. 
3.	The size of feature maps being fed to the first fully connected layer must be less than 10 x 10, while there is no number limitation about the depth.
4.	You may choose any techniques for preventing overfitting. 

In the end of the cell, use `modelA.summary()` to output the model architecture. You may also use `plot_model()` to visualize its architecture.

In [6]:
%%capture
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import layers, regularizers

# Define the input shape
input_shape = (128, 128, 3)

# Define the number of filters for each convolutional layer
filters = [32, 64, 128, 256, 512, 1024, 2048]

# Create the input layer
inputs = Input(shape=input_shape)

# Convolutional layers
x = inputs
for num_filters in filters:
    x = Conv2D(num_filters, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    if num_filters == 256:
        #gender branch
        gender_conv = x
        
# Apply dropout after the convolutional layers
age_conv_drop = Dropout(0.5)(x)

# Flatten the convolutional outputs
age_conv_flat = Flatten()(age_conv_drop)
gender_conv_flat = Flatten()(gender_conv)

#Apply dropout after the flattening layers
age_conv_flat_drop = Dropout(0.5)(age_conv_flat)
gender_conv_flat_drop = Dropout(0.5)(gender_conv_flat)


# Dense layers for age prediction
age_dense = Dense(2048, activation='relu')(age_conv_flat_drop)
age_dense = Dropout(0.5)(age_dense)
age_output = Dense(1, activation='relu', name='age')(age_dense)

# Dense layers for gender prediction
gender_dense = Dense(2048, activation='relu')(gender_conv_flat)
gender_dense = Dropout(0.5)(gender_dense)
gender_output = Dense(1, activation='sigmoid', name='gender')(gender_dense)

# Create the model with multiple outputs
modelA = Model(inputs=inputs, outputs=[age_output, gender_output])

# Print the model summary
modelA.summary()
from tensorflow.keras.utils import plot_model
plot_model(modelA, show_shapes=True)

## STEP3A: Compile and train your model
Compile and train your model here. 
Save your model by `modelA.save(your_model_folder+"age_gender_A.h5")` after training. 

**DON'T use any other name for your model file.** This is because my test code relies on this particular model name. Any other file name will cause problem in the testing stage.

**Save the model with `save()` instead of `save_weights()`.** This is because I will load the model by 

`modelA = load_model(model_folder+"age_gender_A.h5")`. 


In [7]:
from tensorflow.keras.optimizers import Adam


# Compile the model
modelA.compile(optimizer=Adam(learning_rate=0.001),
              loss={'age': 'mean_squared_error', 'gender': 'binary_crossentropy'},
              metrics={'age': 'mae', 'gender': 'accuracy'})

#this extra function is required as we have two outputs, code from Vijayabhaskar, J. (2019) Multi-label image classification Tutorial with Keras ImageDataGenerator.
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(2)])

#calculate step sizes
step_size_train=train_generator.n//train_generator.batch_size
step_size_val=test_generator.n//test_generator.batch_size


#start training using fit_generator as we are using an image generator, 240 epochs = 2 hours with GPU 
history = modelA.fit(
      generator_wrapper(train_generator),
      steps_per_epoch=step_size_train,
      validation_data=generator_wrapper(test_generator),
      validation_steps=step_size_val,
      epochs=250
)


'''# Train the model
history = modelA.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    validation_data=test_generator,
                    validation_steps=len(test_generator))
'''
#save model
model_folder = 'models/'
import os
if not os.path.exists(model_folder):
    os.mkdir(model_folder)
modelA.save(model_folder+"age_gender_A.h5")

Epoch 1/250
31/31 [==============================] - 10s 243ms/step - loss: 4764.1968 - age_loss: 4762.9604 - gender_loss: 1.2380 - age_mae: 41.6221 - gender_accuracy: 0.5169 - val_loss: 109.1517 - val_age_loss: 108.4676 - val_gender_loss: 0.6840 - val_age_mae: 7.7544 - val_gender_accuracy: 0.6339
Epoch 2/250
31/31 [==============================] - 7s 236ms/step - loss: 524.6104 - age_loss: 523.9272 - gender_loss: 0.6832 - age_mae: 17.7729 - gender_accuracy: 0.5721 - val_loss: 459.5237 - val_age_loss: 458.8452 - val_gender_loss: 0.6785 - val_age_mae: 18.8272 - val_gender_accuracy: 0.5949
Epoch 3/250
31/31 [==============================] - 7s 238ms/step - loss: 410.5912 - age_loss: 409.9116 - gender_loss: 0.6797 - age_mae: 15.4738 - gender_accuracy: 0.5762 - val_loss: 746.6598 - val_age_loss: 745.9786 - val_gender_loss: 0.6812 - val_age_mae: 25.3916 - val_gender_accuracy: 0.5525
Epoch 4/250
31/31 [==============================] - 7s 239ms/step - loss: 408.7258 - age_loss: 408.0503 - 

## STEP4A: Draw the learning curves
Draw four figures as follows
1.	The loss of the gender classification over the training and validation set
2.	The accuracy of the gender classification over the training and validation set
3.	The loss of the age estimation over the training and validation set
4.	The MAE of the age estimation over the training and validation set


In [8]:
#
# Add your code here
#


## STEP2B: Build a CNN network based on a pre-trained model 
Choose one existing CNN architecture pre-trained on ImageNet, and fine-tune on this dataset.

The same as required in Model A, **don’t resize the input image size**. **The output layer for the gender branch is set to have only 1 unit**. 

In the end of the cell, use `modelB.summary()` to output the model architecture. You may also use `plot_model()` to visualize its architecture.


In [9]:
#
# Add your code here
#

modelB.summary()
from tensorflow.keras.utils import plot_model
plot_model(modelB, show_shapes=True)

NameError: name 'modelB' is not defined

## STEP3B: Compile and train your model
Compile and train your model here. 
Save your model to `age_gender_B.h5` after training. 

**DON'T use any other name for your model file.** This is because my test code relies on this particular model name. Any other file name will cause problem in the testing stage.

**Also, save the model with `save()` instead of `save_weights()`.** 


In [ ]:
#
# Add your code here
#

modelB.save(your_model_folder+"age_gender_B.h5")

## STEP4B: Draw the learning curve
Draw four figures as follows
1.	The loss of the gender classification over the training and validation set
2.	The accuracy of the gender classification over the training and validation set
3.	The loss of the age estimation over the training and validation set
4.	The MAE of the age estimation over the training and validation set

In [ ]:
#
# Add your code here
#


## STEP5: Evaluate the model on the test set
I will add my test code here to test the two models you trained. The test set will not be available before your submission. 

The metrics for measuring the performance on the test set are:
- age estimation: MAE (Mean Absolute Error)
- gender classification: accuracy


In [ ]:
#
# Don't add code in this cell when submitting this file
#